In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score

In [35]:
# Читаем
# Там разделитель \t и нет имен столбцов
data_train = pd.read_csv('train.txt', delimiter='\t', header=0, names=['name', 'group'])
data_validate = pd.read_csv('test.txt', delimiter='\t', header=0, names=['name'])

# В нижний регистр
data_train['name'] = data_train['name'].str.lower()
#data_train['group'] = data_train['group'].str.lower()

data_validate['name'] = data_validate['name'].str.lower()

data_train

,name,group
0,державка 30531402 mapal,Резцы
1,"кабель кпсввнг-ls 1х2х0,75",Кабельная продукция
2,"трубка электроизоляционная ткр ф16,0мм",Изделия электроустан
3,"лента конвейер 2,1-1000-тк-200-2-5/2",ИзделияРезино-технич
4,соединение быстроразъемное hs-ss-e-0100,Запчасти
...,...,...
23967,"фреза шпоночная ц/х 8,0",Фрезы
23968,кирпич керам полнотел одинарный м200,МатерСтроительные
23969,"клеймо тв спл 122""ф"" вк15",Инструменты
23970,элемент питания saft ls 14250/std 1/2aa,Запчасти


In [36]:
# Удаляем всё, что не буквы и не цифры
#data_train['name'] = data_train['name'].str.replace('[^a-zA-Zа-яА-Я1-9 ]', ' ')
#data_validate['name'] = data_validate['name'].str.replace('[^a-zA-Zа-яА-Я1-9 ]', ' ')
data_train['name'] = data_train['name'].str.replace('[^a-zA-Zа-яА-Я]', ' ')
data_validate['name'] = data_validate['name'].str.replace('[^a-zA-Zа-яА-Я]', ' ')

data_train

,name,group
0,державка mapal,Резцы
1,кабель кпсввнг ls х х,Кабельная продукция
2,трубка электроизоляционная ткр ф мм,Изделия электроустан
3,лента конвейер тк,ИзделияРезино-технич
4,соединение быстроразъемное hs ss e,Запчасти
...,...,...
23967,фреза шпоночная ц х,Фрезы
23968,кирпич керам полнотел одинарный м,МатерСтроительные
23969,клеймо тв спл ф вк,Инструменты
23970,элемент питания saft ls std aa,Запчасти


In [38]:
# Удаляем слова короче определенного количества символов

min_len = 3
data_train['name'] = data_train['name'].str.split().map(lambda sl: " ".join(s for s in sl if len(s) >= min_len))
data_validate['name'] = data_validate['name'].str.split().map(lambda sl: " ".join(s for s in sl if len(s) >= min_len))

data_train

,name,group
0,державка mapal,Резцы
1,кабель кпсввнг,Кабельная продукция
2,трубка электроизоляционная ткр,Изделия электроустан
3,лента конвейер,ИзделияРезино-технич
4,соединение быстроразъемное,Запчасти
...,...,...
23967,фреза шпоночная,Фрезы
23968,кирпич керам полнотел одинарный,МатерСтроительные
23969,клеймо спл,Инструменты
23970,элемент питания saft std,Запчасти


In [40]:
# Попробуем кириллицу оттранслитить

dictionary = {'а':'a','б':'b','в':'v','г':'g','д':'d','е':'e','ё':'yo',
      'ж':'zh','з':'z','и':'i','й':'i','к':'k','л':'l','м':'m','н':'n',
      'о':'o','п':'p','р':'r','с':'s','т':'t','у':'u','ф':'f','х':'h',
      'ц':'c','ч':'ch','ш':'sh','щ':'sch','ъ':'','ы':'y','ь':'','э':'e',
      'ю':'u','я':'ya'}

for cyr, lat in dictionary.items():
    #data_train['name'] = data_train['name'].str.replace(cyr, lat)
    #data_validate['name'] = data_validate['name'].str.replace(cyr, lat)
    pass
    
data_train['name']

0                         державка mapal
1                         кабель кпсввнг
2         трубка электроизоляционная ткр
3                         лента конвейер
4             соединение быстроразъемное
                      ...               
23967                    фреза шпоночная
23968    кирпич керам полнотел одинарный
23969                         клеймо спл
23970           элемент питания saft std
23971            труба бшхт прецизионная
Name: name, Length: 23972, dtype: object

In [41]:
# Проверяем, не появилось ли пустых строк. Спойлер - появились
print(data_train[data_train['name'] == ''])
print('---------------------------------------------')
print(data_validate[data_validate['name'] == ''])

     name                 group
1535       Периклазоуглеродисты
---------------------------------------------
     name
1276     
1576     


In [42]:
# Series to List
# Не будем удлинять, юзнем потом padsequence
text_train = data_train['name'].tolist()
text_validate = data_validate['name'].tolist()

# Длина самой длинной строки в символах
max_str_char_len = data_train['name'].str.len().max()
print('Максимальная длина в символах:', max_str_char_len)

# Длина самой длинной строки в словах
max_str_len = max([len(str_i.split()) for str_i in text_train])
print('Максимальная длина в словах:', max_str_len) 

Максимальная длина в символах: 40
Максимальная длина в словах: 7


In [43]:
# Превращаем тексты в последовательности чисел, соответствующих словам
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_words = 10000
tokenizer = Tokenizer(num_words=max_words, char_level=False, filters='')  # токенизируем на уровне слов
tokenizer.fit_on_texts(data_train['name'].tolist())  # формируем токены на основе частотности в нашем тексте

# Здесь каждую строку в лист чисел
X_train = tokenizer.texts_to_sequences(text_train)
X_validate = tokenizer.texts_to_sequences(text_validate)

# Выравниваем длину, добавляя нули первыми
X_train = pad_sequences(X_train, maxlen=max_str_len)
X_validate = pad_sequences(X_validate,  maxlen=max_str_len)

# Количество уникальных символов, которые мы тут имеем
num_words = len(tokenizer.word_index)
print('Количество уникальных слов:', num_words)
X_train.shape

Количество уникальных слов: 10150


(23972, 7)

In [44]:
# Заменим категории на другой OHE
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_train = enc.fit_transform(data_train['group'].to_numpy().reshape(-1, 1)).toarray()
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [45]:
# Количество уникальных классов
n_categories = enc.categories_[0].shape[0]
print(n_categories)

96


In [46]:
# Попробуем простенькую модельку
import keras as k

model = k.models.Sequential()
#model.add(k.layers.Input((max_str_len, num_words)))
model.add(k.layers.Embedding(num_words+1, 32, input_length = max_str_len))
model.add(k.layers.GRU(32, return_sequences=True))
#model.add(k.layers.GRU(32, return_sequences=True))
#model.add(k.layers.GRU(32, return_sequences=True))
model.add(k.layers.GRU(n_categories))
model.add(k.layers.Dense(n_categories, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 7, 32)             324832    
_________________________________________________________________
gru_35 (GRU)                 (None, 7, 32)             6336      
_________________________________________________________________
gru_36 (GRU)                 (None, 96)                37440     
_________________________________________________________________
dense_11 (Dense)             (None, 96)                9312      
Total params: 377,920
Trainable params: 377,920
Non-trainable params: 0
_________________________________________________________________


In [48]:
# Учимся
history = model.fit(X_train, y_train, batch_size=200, epochs=10, validation_split = 0.1)

Epoch 1/10
108/108 [==============================] - 2s 21ms/step - loss: 0.4809 - accuracy: 0.8691 - val_loss: 1.3652 - val_accuracy: 0.7148
Epoch 2/10
108/108 [==============================] - 2s 20ms/step - loss: 0.4512 - accuracy: 0.8740 - val_loss: 1.3540 - val_accuracy: 0.7331
Epoch 3/10
108/108 [==============================] - 2s 20ms/step - loss: 0.4231 - accuracy: 0.8798 - val_loss: 1.3786 - val_accuracy: 0.7281
Epoch 4/10
108/108 [==============================] - 2s 20ms/step - loss: 0.4027 - accuracy: 0.8819 - val_loss: 1.3885 - val_accuracy: 0.7331
Epoch 5/10
108/108 [==============================] - 2s 20ms/step - loss: 0.3857 - accuracy: 0.8848 - val_loss: 1.3909 - val_accuracy: 0.7335
Epoch 6/10
108/108 [==============================] - 2s 20ms/step - loss: 0.3665 - accuracy: 0.8886 - val_loss: 1.4012 - val_accuracy: 0.7335
Epoch 7/10
108/108 [==============================] - 2s 21ms/step - loss: 0.3535 - accuracy: 0.8911 - val_loss: 1.4300 - val_accuracy: 0.7348

In [ ]:
# Записываем веса в файл
model.save('.\\on_words_gru_saved')

In [ ]:
# Читаем веса из файла
model = k.models.load_model('.\\on_words_gru_saved')